<a href="https://colab.research.google.com/github/shininglight99/ellison-archive/blob/master/10Year_US_treasury_Note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install QuantLib-Python 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests
from bs4 import BeautifulSoup
import QuantLib as ql

In [3]:
def GET_DATE():
  req = requests.get("https://www.marketwatch.com/investing/bond/tmubmusd10y?countrycode=bx")
  html = req.text
  soup = BeautifulSoup(html, 'html.parser')
  data = soup.find("span", class_="WSJBase--card__timestamp--2xDXNOQK")
  date = data.text
  date = datetime.datetime.strptime(date, "%m/%d/%y").date()
  return date

In [4]:
import re
import requests
from datetime import datetime
import pandas as pd

def get_quote(reference_date):
    if type(reference_date) != datetime:
        reference_date = datetime.strptime(reference_date, "%Y-%m-%d")
    maturity = ["01m", "03m", "06m", "01y", "02y", "03y", "05y", "07y", "10y", "30y"]
    bond_table = pd.DataFrame(columns = ["Maturity Date","Price" , "Coupon Rate"], \
                             index = maturity)
    for mat in maturity:
        url = f"https://www.marketwatch.com/investing/bond/tmubmusd{mat}?countrycode=bx"
        data = requests.get(url)
        _, _, _, price, _, _, coupon_rate, maturity_date = \
                re.findall(r"<span class=\"primary \">(.*?)</span>\n", data.text)
        maturity_date = datetime.strptime(maturity_date, "%b %d, %Y")#\
                                                            #.strftime("%Y-%m-%d")
        if mat in ("01m", "03m", "06m", "01y"):
            numer, denom = price.split("/")
            discount_yield = int(numer) / float(denom)
            n = (maturity_date - reference_date).days
            price = 100 / (1 + n * discount_yield / 365)
        else:
            price1, fraction = price.split()[0], price.split()[1]
            numer, denom = fraction.split("/")[0], fraction.split("/")[1]
            price = int(price1) + int(numer) / float(denom)
        
        maturity_date = maturity_date.strftime("%Y-%m-%d")
        coupon_rate = float(coupon_rate[:-1]) / 100
        bond_table.loc[mat] = [maturity_date, price, coupon_rate]
    return bond_table

get_quote(datetime.today())

,Maturity Date,Price,Coupon Rate
01m,2021-03-02,99.5059,0
03m,2021-04-29,98.5321,0
06m,2021-07-29,95.628,0
01y,2022-01-27,91.5361,0
02y,2023-01-31,100,0.00125
03y,2024-01-15,99.25,0.00125
05y,2026-01-31,99.2188,0.00375
07y,2028-01-31,99.25,0.0075
10y,2030-11-15,98,0.00875
30y,2050-11-15,94.2812,0.01625


In [5]:
#Test
def TREASURY_CURVE(date, quote):

  tbill = quote[0:4]
  tbond = quote[4:]

In [12]:
#Set Evaluation Date

eval_date = ql.Date(date.day, date.month, date.year)
ql.Settings.instance().evaluationDate = eval_date

NameError: ignored

In [ ]:
#Set Market Conventions
calendar = ql.UnitedStates()
convention = ql.ModifiedFollowing
day_counter = ql.ActualActual()
end_of_month = True
fixing_days = 1
face_amount = 100
coupon_frequency = ql.Period(ql.Semiannual)

In [ ]:
#Construct Treasury Bill Helpers
bill_helpers = [ql.DepositRateHelper(ql.QuoteHandle(ql.SimpleQuote(r/100.0)),
                                     ql.Period(m, ql.Days),
                                     fixing_days,
                                     calendar,
                                     convention,
                                     end_of_month,
                                     day_counter)
for r, m in zip(tbill['price'], tbill['days'])]